<a href="https://colab.research.google.com/github/MASAYA-KINOSHITA/first-Repository/blob/main/%E6%B0%91%E6%B3%8A%E3%82%B5%E3%83%BC%E3%83%93%E3%82%B9%E3%81%AE%E5%AE%BF%E6%B3%8A%E6%96%99%E9%87%91%E4%BA%88%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#from モジュール名 import クラス名（もしくは関数名や変数名）
import pandas as pd
from pandas import DataFrame
import numpy as np

In [2]:
#データの読み込み
train_df = pd.read_csv("/content/drive/MyDrive/民泊サービスの宿泊料金予測/train/train_data.csv")
test_df = pd.read_csv("/content/drive/MyDrive/民泊サービスの宿泊料金予測/test/test_data.csv")
submit_df = pd.read_csv("/content/drive/MyDrive/民泊サービスの宿泊料金予測/提出用ファイル/submission.csv")
station_list_df = pd.read_csv("/content/drive/MyDrive/民泊サービスの宿泊料金予測/train/station_list.csv")
print(train_df.shape)
print(test_df.shape)

(9990, 13)
(4996, 12)


In [3]:
#データの確認
train_df.head(10)

,id,name,host_id,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,availability_365,y
0,1,KiyosumiShirakawa 3min|★SkyTree★|WIFI|Max4|Tre...,242899459,Koto Ku,35.68185,139.80310,Entire home/apt,1,55,2020-04-25,2.21,173,12008
1,2,Downtown Tokyo Iriya next to Ueno,308879948,Taito Ku,35.72063,139.78536,Entire home/apt,6,72,2020-03-25,2.11,9,6667
2,3,"Japan Style,Private,Affordable,4min to Sta.",300877823,Katsushika Ku,35.74723,139.82349,Entire home/apt,1,18,2020-03-23,3.46,288,9923
3,4,4 min to Shinjuku Sta. by train / 2 ppl / Wi-fi,236935461,Shibuya Ku,35.68456,139.68077,Entire home/apt,1,2,2020-04-02,1.76,87,8109
4,5,LICENSED SHINJUKU HOUSE: Heart of the action!,243408889,Shinjuku Ku,35.69840,139.70467,Entire home/apt,1,86,2020-01-30,2.00,156,100390
5,6,"Quiet residential Shinjuku area,8 min on foot Sta",247956261,Shinjuku Ku,35.71273,139.69687,Entire home/apt,2,30,2020-03-29,5.20,335,8404
6,7,5mins walk to sta. /FamilyMat /double room liuqm1,314405566,Arakawa Ku,35.73180,139.79445,Entire home/apt,1,8,2020-04-15,1.69,111,5811
7,8,花川戸(竹)池袋から600米高端民宿(空港送迎ｻｰﾋﾞｽあり),256531617,Toshima Ku,35.73665,139.70821,Entire home/apt,1,7,2020-02-02,0.93,134,19938
8,9,"东京精品民宿5,近新宿､渋谷､池袋､銀座､秋葉原､上野",114995297,Toshima Ku,35.73698,139.73243,Private room,1,12,2020-04-27,0.88,89,10216
9,10,【2/F】Asakusabashi Building | for 6-9 ppl,260891041,Taito Ku,35.70020,139.78538,Entire home/apt,2,74,2020-01-26,1.48,0,24826


In [4]:
"""
各カラムの処理をメモする。
・id, host_idは削除する。
・緯度経度は主要駅からの距離の特徴量とする。
・name,last_reviewは一旦削除する。
・reviews_per_monthの欠損については処理せずに進める。
・neighbourhood,room_typeはラベルエンコーディングを行う。
"""

'\n各カラムの処理をメモする。\n・id, host_idは削除する。\n・緯度経度は主要駅からの距離の特徴量とする。\n・name,last_reviewは一旦削除する。\n・reviews_per_monthの欠損については処理せずに進める。\n・neighbourhood,room_typeはラベルエンコーディングを行う。\n'

In [5]:
#目的変数のみのリストを作成
train_y=train_df["y"]

#目的変数、カテゴリ変数、緯度、経度を除いたリストを作成
train_x=train_df.drop(["id","host_id","y","neighbourhood","room_type","latitude","longitude","name"],axis=1)
test_x=test_df.drop(["id","host_id","neighbourhood","room_type","latitude","longitude","name"],axis=1)

In [6]:
# 各カラムの種類の数を確認
for colname in train_x.columns:
  lencol = len(train_x[colname].unique())
  print(colname,lencol)
  

minimum_nights 30
number_of_reviews 261
last_review 547
reviews_per_month 595
availability_365 366


In [7]:
#availability_に366以上の値が入っていないかチェックする。
reqd_Index = train_x[train_x['availability_365']>=366].index.tolist()
print(reqd_Index)

[]


In [8]:
#minimum_nightsに366以上の値が入っていないかチェックする。
reqd_Index = train_x[train_x['minimum_nights']>=366].index.tolist()
print(reqd_Index)

[]


In [9]:
#train_dfから緯度経度を取得
train_geo=train_df.loc[:,['latitude','longitude']]
test_geo=test_df.loc[:,['latitude','longitude']]
train_geo.head()

,latitude,longitude
0,35.68185,139.80310
1,35.72063,139.78536
2,35.74723,139.82349
3,35.68456,139.68077
4,35.69840,139.70467


In [10]:
#station_listから東京駅、渋谷駅、新宿駅の行を取得する。
print (station_list_df.loc[(station_list_df['station_name'] == "東京")|(station_list_df['station_name'] == "新宿")|(station_list_df['station_name'] == "渋谷")])

    station_name   longitude   latitude
89            新宿  139.698812  35.688690
90            新宿  139.699187  35.690163
91            新宿  139.699574  35.691435
92            新宿  139.700464  35.689729
93            新宿  139.700711  35.692350
94            渋谷  139.700872  35.658700
95            渋谷  139.701000  35.659066
96            渋谷  139.701238  35.658871
97            渋谷  139.702148  35.658020
98            渋谷  139.702417  35.659545
160           東京  139.764708  35.681753
162           東京  139.766103  35.681391


In [11]:
#複数存在するのでそれぞれの平均を取る。
tokyo_df=station_list_df.loc[(station_list_df["station_name"]=="東京")]
tokyo_df=tokyo_df.mean()
shibuya_df=station_list_df.loc[(station_list_df["station_name"]=="渋谷")]
shibuya_df=shibuya_df.mean()
shinjuku_df=station_list_df.loc[(station_list_df["station_name"]=="新宿")]
shinjuku_df=shinjuku_df.mean()
print(tokyo_df)
print(shibuya_df)
print(shinjuku_df)

longitude    139.765405
latitude      35.681572
dtype: float64
longitude    139.701535
latitude      35.658840
dtype: float64
longitude    139.699750
latitude      35.690473
dtype: float64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  import sys


In [12]:
#主要駅の緯度経度のDataFrameを作成
list2 = [["東京駅",  35.681572,  139.765405], ["渋谷駅", 35.658840,  139.701535],["新宿駅", 35.690473, 139.699750]]
columns = ['location', 'latitude', 'longitude']
locations = pd.DataFrame(data=list2,columns=columns)
locations.head()

,location,latitude,longitude
0,東京駅,35.681572,139.765405
1,渋谷駅,35.658840,139.701535
2,新宿駅,35.690473,139.699750


In [13]:
#geopyを用いて各駅からの距離を算出
from geopy.distance import geodesic

for i in range(len(locations)) :
  train_geo[locations['location'][i]] = train_geo.apply(lambda x:geodesic((x["latitude"], x["longitude"]), (locations['latitude'][i], locations['longitude'][i])), axis=1)

for i in range(len(locations)) :
  test_geo[locations['location'][i]] = test_geo.apply(lambda x:geodesic((x["latitude"], x["longitude"]), (locations['latitude'][i], locations['longitude'][i])), axis=1)

train_geo.head()

,latitude,longitude,東京駅,渋谷駅,新宿駅
0,35.68185,139.80310,3.412464225940314 km,9.543267505960632 km,9.403993297313582 km
1,35.72063,139.78536,4.694873334968289 km,10.226026871003931 km,8.439177659275503 km
2,35.74723,139.82349,8.983122214657186 km,14.76468243897736 km,12.84581105420383 km
3,35.68456,139.68077,7.668589495082938 km,3.4173051020815897 km,1.8390374164137462 km
4,35.69840,139.70467,5.805861771594083 km,4.398468474188946 km,0.985834329272876 km


In [15]:
#train_geoとtrain_xを結合
train_x=pd.concat([train_x, train_geo],axis=1)
test_x=pd.concat([test_x, test_geo],axis=1)

In [16]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [ ]:
# xgboostの実装
# -----------------------------------
import xgboost as xgb
from sklearn.metrics import log_loss

# 特徴量と目的変数をxgboostのデータ構造に変換する
dtrain = xgb.DMatrix(train_x, label=train_y)
dvalid = xgb.DMatrix(va_x,label=va_y)
dtest = xgb.DMatrix(test_x)

In [ ]:
# 学習データとバリデーションデータのスコアのモニタリング
# -----------------------------------
# モニタリングをloglossで行い、アーリーストッピングの観察するroundを20とする
params = {
    #'booster': 'gbtree',
    'objective': 'reg:squarederror',
    #'eta': 0.1,
    #'gamma': 0.0,
    #'alpha': 0.0,
    #'lambda': 1.0,
    #'min_child_weight': 1,
    #'max_depth': 5,
    #'subsample': 0.8,
    #'colsample_bytree': 0.8,
    'silent':1,
    'random_state': 71,
    #上記で取得したパラメータ
}
num_round = 5000
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(params, dtrain, num_round, evals=watchlist,
                  early_stopping_rounds=20)


In [ ]:
# 構築したモデルの保存
import pickle
file = '/content/drive/MyDrive/民泊サービスの宿泊料金予測/モデル/xgboost_notuning1.pkl'
pickle.dump(model, open(file, 'wb'))

# 学習済みモデルを削除
del model

In [ ]:
# 構築したモデルの読み込み
import pickle
filename = '/content/drive/MyDrive/民泊サービスの宿泊料金予測/モデル/xgboost_notuning1.pkl'
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
id_df=submission_df.iloc[:,0]
id_df.head()

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64

In [ ]:
pred=loaded_model.predict(dtest,ntree_limit=loaded_model.best_ntree_limit)

In [ ]:
pred_df = pd.DataFrame(pred)
pred_df.head()

,0
0,1.702089e-07
1,1.931730e-07
2,1.702089e-07
3,1.702089e-07
4,1.931730e-07


In [ ]:
output_df=pd.concat([id_df, pred_df],axis=1)
output_df.columns=['id','y']
output_df.head()

,id,Anomaly
0,0,1.702089e-07
1,1,1.931730e-07
2,2,1.702089e-07
3,3,1.702089e-07
4,4,1.931730e-07


In [ ]:
output_df.to_csv('/content/drive/MyDrive/民泊サービスの宿泊料金予測/提出用ファイル/xgboost_notuning1.csv',index=False)